<a href="https://colab.research.google.com/github/bigirimanainnocent12/Code/blob/main/CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import date
import os
import pandas as pd
import time
import numpy as np
from tqdm import tqdm
from rich import print

def traiter_fichiers(dossier_fichiers, dossier_sauvegarde, dossier_fermenteur):
    # Mesurer le temps de début
    start_time = time.time()

    # Liste des encodages possibles
    encodages = ['utf-8', 'latin-1', 'cp1252', 'ascii', 'utf-16', 'utf-32']

    # Créer des dossiers pour sauvegarder les fichiers Excel
    os.makedirs(dossier_sauvegarde, exist_ok=True)
    os.makedirs(dossier_fermenteur, exist_ok=True)  # Créer le dossier Fermenteur

    # Obtenir la liste des fichiers texte dans le dossier
    fichiers_texte = os.listdir(dossier_fichiers)

    # Parcourir tous les fichiers texte dans le dossier avec une barre de progression
    for fichier_texte in tqdm(fichiers_texte, desc="Traitement des fichiers"):
        chemin_fichier = os.path.join(dossier_fichiers, fichier_texte)

        for encodage in encodages:
            try:
                with open(chemin_fichier, "r", encoding=encodage) as fichier:
                    contenu = fichier.read()
                print(f"Fichier '{fichier_texte}' lu avec l'encodage : {encodage}")

                df = pd.DataFrame([x.split('\t') for x in contenu.split('\n') if x])
                separateur=len(df.columns)
                h = f"{os.path.splitext(fichier_texte)[0]}.xlsx"
                database = 'Lallemand_oenologie'
                server = 'localhost\\SQLEXPRESS'  # Nom du serveur correct
               # Établir la connexion
                connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
                conn = pyodbc.connect(connection_string)
                print("Connexion réussie avec succes!")
                cursor = conn.cursor()

                if separateur==13:

                   # Stocker le nom du fichier Excel dans la variable h

                   print(f"Nom du fichier Excel en cours : {h}")

                   Fermenteur = df.loc[:, 0][1].split('=')[1]
                   date_experience = df.loc[:, 0][df[df[0].str.startswith('Début')].index[0]].split('=')[1]
                   Souche = df.loc[:, 0][df[df[0].str.startswith('Souche')].index[0]].split('=')[1]
                   Milieu = df.loc[:, 0][df[df[0].str.startswith('Milieu')].index[0]].split('=')[1]
                   volume = "Salle"
                   dicti = {'Fermenteur': Fermenteur, 'date_experience': date_experience, 'Souche': Souche, 'Milieu': Milieu, 'volume': volume}
                   data = pd.DataFrame(dicti, index=[0])
                   data['date_experience'] = pd.to_datetime(data['date_experience'], format='%d/%m/%Y %H:%M')
                   data['Code'] = 'F' + data['Fermenteur'].astype(str) + '_' + data['date_experience'].astype(str)+'_' + data['Souche'].astype(str) +'_' + data['Milieu'].astype(str)
                   valeurs_a_chercher = ["[Données]", "[Donn�es]"]
                   df = df.loc[:,][df.loc[df.loc[:, 0].isin(valeurs_a_chercher)].index[0] + 1:]
                   nouvelles_colonnes = df.iloc[0, :len(df.columns)].tolist()
                   df.columns = nouvelles_colonnes
                   # Supprimer la première ligne (qui a été utilisée comme noms de colonnes)
                   df = df.iloc[1:]

                   # Réinitialiser l'index
                   df = df.reset_index(drop=True)
                   df['Tps inject.'] = df['Tps inject.'].str.replace(',', '.').astype(int)

                   max=df['Tps inject.'].max()
                   df.drop(columns=['T°C', 'Poids', 'Volume', 'FinPhas', 'Vinst', 'Csg V', 'V5', 'Acc G','Tps inject.'], inplace=True)

                   df['Code']=data['Code'][0]
                   df.rename(columns={'Csg T': 'Csg_T'}, inplace=True)
                   df['Temps'] = df['Temps'].str.replace(',', '.').astype(float)
                   df['Csg_T'] = df['Csg_T'].str.replace(',', '.').astype(float)
                   df['CO2']=df['CO2'].str.replace(',', '.').astype(float)
                   df['V11']=df['V11'].str.replace(',', '.').astype(float)
                   df.rename(columns={'V11': 'V'}, inplace=True)
                   data['Csg_T']=int(df['Csg_T'].iloc[0])
                   data.drop(columns=['Fermenteur','date_experience'],inplace=True)
                   df.drop(columns='Csg_T',inplace=True)

                   # Traitement des ecarts entre deux observations de la colonne temps
                   taille=len(df)
                   compteur=0
                   for i in np.arange(taille-1):
                      z=df['Temps'].iloc[i+1]-df['Temps'].iloc[i]
                      if z>70:
                         compteur=compteur+1
                   # Traitement des valeurs manquantes
                   S = 0
                   for i in np.arange(len(df)):
                      if df['CO2'].isna().iloc[i] == True:
                         S = S + 1

                   H=0
                   for i in np.arange(len(df)):
                      if df['V'].isna().iloc[i]==True:
                         H=H+1
                   if df['CO2'].iloc[-2]>df['CO2'].iloc[-1]:
                     df=df[:len(df) - 1]
                   else:
                       df=df
                   enreg=0
                   if S==0 and compteur==0 and len(df) > 100 and df['V'].iloc[-1]<0.1 and df['CO2'].max()>60 and df['V'].iloc[12]<0.2 and max==0 and H==0:
                       # Chemin complet pour sauvegarder le fichier Excel
                       chemin_excel = os.path.join(dossier_sauvegarde, h)
                       df.to_excel(chemin_excel, index=False)
                       chemin_fermenteur = os.path.join(dossier_fermenteur, h)
                      # data.to_excel(chemin_fermenteur, index=False)
                       enreg=1
                       print(f"Les DataFrames ont été sauvegardés sous format Excel dans '{chemin_excel}' et '{chemin_fermenteur}'.")
                   else:
                       enreg=2
                       print("[Le fichier ne respecte pas toutes les conditions requises pour être sauvegarder]")

                   if enreg==1:
                      print("Données insérées avec succès!")
                       # Insérer les données dans la table SQL
                      for index, row in data.iterrows():
                           cursor.execute('''
                                        INSERT INTO Fermentation (Souche, Milieu, volume, Code,Csg_T)
                                        VALUES (?, ?, ?, ?, ?)
                                         ''',
                                       row['Souche'], row['Milieu'], row['volume'], row['Code'],row['Csg_T'])
                   # Valider la transaction
                   conn.commit()
                   if enreg==1:

                       # Insérer les données dans la table SQL
                      for index, row in df.iterrows():
                           cursor.execute('''
                                        INSERT INTO donnee (Temps, CO2,V, Code)
                                        VALUES (?, ?, ?, ?)
                                         ''',
                                       row['Temps'], row['CO2'], row['V'], row['Code'])
                   # Valider la transaction






                   conn.commit()
                   # Fermer le curseur et la connexion
                   cursor.close()
                   conn.close()
                   print('Connexion ferme avec succes!')


                else:
                  Fermenteur = df.loc[:, 0][1].split('=')[1]
                  date_experience = df.loc[:, 0][df[df[0].str.startswith('Début')].index[0]].split('=')[1]
                  Souche = df.loc[:, 0][df[df[0].str.startswith('Souche')].index[0]].split('=')[1]
                  Milieu = df.loc[:, 0][df[df[0].str.startswith('Milieu')].index[0]].split('=')[1]
                  volume = "Robot"
                  dicti = {'Fermenteur': Fermenteur, 'date_experience': date_experience, 'Souche': Souche, 'Milieu': Milieu, 'volume': volume}
                  data = pd.DataFrame(dicti, index=[0])
                  data['date_experience'] = pd.to_datetime(data['date_experience'], format="%Y/%m/%d %H:%M")
                  data['Code'] =data['Fermenteur'].astype(str) + '_' + data['date_experience'].astype(str)+'_' + data['Souche'].astype(str) +'_' + data['Milieu'].astype(str)
                  valeurs_a_chercher=["[Données]","[Donn�es]"]
                  df=df.loc[:,][df.loc[df.loc[:, 0].isin(valeurs_a_chercher)].index[0]+1:]
                  nouvelles_colonnes = df.iloc[0, :10].tolist()
                  remaining_columns = df.columns[10:].tolist()
                  df.columns = nouvelles_colonnes + remaining_columns
                  df = df.iloc[1:]
                  df=df.iloc[1:].reset_index(drop=True)
                  df['Code']=data['Code'][0]
                  df.drop(["Prélèvement","V","V3",'Volume','Poids','T°C'],inplace=True,axis=1)
                  df.rename(columns={'Csg T': 'Csg_T'}, inplace=True)
                  df.rename(columns={'V5': 'V'}, inplace=True)
                  df['Temps'] = df['Temps'].str.replace(',', '.').astype(float)
                  df['Csg_T'] = df['Csg_T'].str.replace(',', '.').astype(float)
                  df['CO2']=df['CO2'].str.replace(',', '.').astype(float)
                  df['V']=df['V'].str.replace(',', '.').astype(float)
                  data['Csg_T']=int(df['Csg_T'].iloc[0])
                  data.drop(columns=['Fermenteur','date_experience'],inplace=True)
                  df.drop(columns='Csg_T',inplace=True)
                  taille=len(df)
                  compteur=0
                  for i in np.arange(taille-1):
                     z=df['Temps'].iloc[i+1]-df['Temps'].iloc[i]
                     if z>70:
                        compteur=compteur+1
                  z=0
                  for i in np.arange(len(df)):
                     if df['V'].isna().iloc[i]==True and df['CO2'].isna().iloc[i]==True and df['Temps'].isna().iloc[i]==True:
                        z=z+1
                  if df['CO2'].iloc[-2]>df['CO2'].iloc[-1]:
                     df=df[:len(df) - 1]
                  else:
                       df=df
                  enreg=0

                  if z==0 and compteur==0 and len(df) > 50 and df['CO2'].max()>60 and df['V'].iloc[-1]<0.1 and df['V'].iloc[4]<0.2:
                     chemin_excel = os.path.join(dossier_sauvegarde, h)
                     df.to_excel(chemin_excel, index=False)
                     chemin_fermenteur = os.path.join(dossier_fermenteur, h)
                     data.to_excel(chemin_fermenteur, index=False)
                     enreg=1
                     print(f"Les DataFrames ont été sauvegardés sous format Excel dans '{chemin_excel}' et '{chemin_fermenteur}'.")
                  else:
                    enreg=2
                    print("[Le fichier ne respecte pas toutes les conditions requises pour être sauvegarder]")
                  if enreg==1:
                    print("Données insérées avec succès!")
                    # Insérer les données dans la table SQL
                    for index, row in data.iterrows():
                           cursor.execute('''
                                        INSERT INTO Fermentation (Souche, Milieu, volume, Code,Csg_T)
                                        VALUES (?, ?, ?, ?, ?)
                                         ''',
                                       row['Souche'], row['Milieu'], row['volume'], row['Code'],row['Csg_T'])
                        # Valider la transaction
                  conn.commit()
                  if enreg==1:
                      # Insérer les données dans la table SQL
                      for index, row in df.iterrows():
                           cursor.execute('''
                                        INSERT INTO donnee (Temps, CO2,V, Code)
                                        VALUES (?, ?, ?, ?)
                                         ''',
                                       row['Temps'], row['CO2'], row['V'], row['Code'])
                  # Valider la transaction
                  conn.commit()
                  # Fermer le curseur et la connexion
                  cursor.close()
                  conn.close()
                  print('Connexion ferme avec succes!')


                break  # Sortir de la boucle si la lecture est réussie

            except UnicodeDecodeError:
                print(f"Échec de la lecture avec l'encodage : {encodage}")
            except FileNotFoundError:
                print(f"Erreur : Le fichier '{chemin_fichier}' n'existe pas.")
                break
            except Exception as e:
                print(f"Une erreur s'est produite avec l'encodage {encodage} : {e}")


    end_time = time.time()
    print(f"Temps total de traitement : {(end_time - start_time)/60} Minutes")